# Statistiques descriptives des données CAF/EMS

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_stbg = pd.read_excel(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/données caf/CALCUL_STBG_anonymisé_V2.xlsx"
)
df_ems = pd.read_excel(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/données caf/CALCUL_EMS_anonymisé_V2.xlsx"
)
df = pd.concat([df_stbg, df_ems])

In [ ]:
pd.options.display.float_format = lambda v: "{:_.4f}".format(v).replace("_", " ")

In [ ]:
(df_stbg.shape, df_ems.shape)

In [ ]:
percentiles = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]

In [ ]:
df.describe(percentiles=percentiles)

In [ ]:
(df["QFEMS"] > 820).sum() / df.shape[0]

In [ ]:
(df["MONTANT QF CNAF"] > 820).sum() / df.shape[0]

In [ ]:
0.35904097846355065 / 0.2969237431179688

In [ ]:
ddf = df.groupby("SITUATION FAMILIALE").describe(percentiles=percentiles).transpose()
ddf

In [ ]:
(df == 0).sum() / (df == 0).count()

In [ ]:
import seaborn

In [ ]:
l = 2500
df["MONTANT QF CNAF C"] = np.minimum(df["MONTANT QF CNAF"], l)
df["QFEMS C"] = np.minimum(df["QFEMS"], l)
seaborn.scatterplot(
    df[(df["QFEMS C"] != 0) * ~((df["QFEMS C"] == l) * (df["MONTANT QF CNAF C"] == l))],
    x="MONTANT QF CNAF C",
    y="QFEMS C",
    hue="SITUATION FAMILIALE",
    s=1,
    alpha=0.1,
).set(xlim=[0, l], ylim=[0, l])

In [ ]:
l = 4000
df["MONTANT QF CNAF C"] = np.minimum(df["MONTANT QF CNAF"], l)
df["QFEMS C"] = np.minimum(df["QFEMS"], l)

for g, gdf in df[
    (df["QFEMS C"] != 0) * (df["QFEMS C"] < l) * (df["MONTANT QF CNAF C"] < l)
].groupby("SITUATION FAMILIALE"):
    display(g)
    display(gdf.describe())
    p = seaborn.jointplot(
        gdf,
        x="MONTANT QF CNAF C",
        y="QFEMS C",
        kind="hist",
        xlim=[0, l],
        ylim=[0, l],
        bins=50,
    )
    p.fig.suptitle(g)

In [ ]:
l = 1000
df["MONTANT QF CNAF C"] = np.minimum(df["MONTANT QF CNAF"], l)
df["QFEMS C"] = np.minimum(df["QFEMS"], l)
p = seaborn.jointplot(
    df[(df["QFEMS C"] != 0) * ~((df["QFEMS C"] == l) * (df["MONTANT QF CNAF C"] == l))],
    x="MONTANT QF CNAF C",
    y="QFEMS C",
    kind="hist",
    xlim=[0, l],
    ylim=[0, l],
)  # .set(title="ok")
p

In [ ]:
l = 1000
seaborn.jointplot(
    df[(df["MONTANT QF CNAF"] < l) * (df["QFEMS"] < l) * (df["QFEMS"] != 0)],
    x="MONTANT QF CNAF",
    y="QFEMS",
    kind="hist",
    xlim=[0, l],
    ylim=[0, l],
)

In [ ]:
import seaborn

seaborn.ecdfplot(df).set(title="Distribution cumulatives des QF", xlim=[0, 4000])

## QF EMS nuls

In [ ]:
df["MONTANT QF CNAF"][df["QFEMS"] == 0].describe(percentiles=percentiles)

In [ ]:
df["MONTANT QF CNAF"][df["QFEMS"] == 248].plot.hist(
    bins=10, title="Distribution des QF CAF pour les QF EMS nuls"
)

In [ ]:
c = df["QFEMS"] == 0
df["MONTANT QF CNAF"][c].plot.hist(
    bins=50, title="Distribution des QF CAF pour les QF EMS nuls"
)

In [ ]:
seaborn.ecdfplot(df["MONTANT QF CNAF"][c]).set(xlim=[0, 800])

In [ ]:
l = 0.002
v = 271
c = ((df["MONTANT QF CNAF"] - v) / v).abs() <= l

In [ ]:
df["QFEMS"][c].sort_values()  #'QFEMSE ESTIME')

In [ ]:
df["QFEMS"][c].plot.hist(bins=10, title="Distribution des QF CAF pour les QF EMS nuls")

In [ ]:
seaborn.ecdfplot(df["QFEMS"][c]).set(xlim=[0, 800])

## Lien QF CAF / QF EMS

In [ ]:
lims = [0, 4000]
df.plot.scatter(
    x="MONTANT QF CNAF",
    y="QFEMS",
    xlim=lims,
    ylim=lims,
    s=0.05,
    title="QFEMS = f(QF CAF)",
)

La diagonale (0,0) (4000, 4000) sépare les personnes dont le QF EMS est supérieur au CAF CAF (au dessus) et celles dont le QF EMS est inférieur (en dessous).

Les pentes qui apparaissent correspondent au rapport entre le nombre de parts CAF et celui du fisc. En effet, on peut dire
QF CAF = (R(essources) + P(restations))/ NbPartsCAF
QF EMS = R / NbPartsFiscales
QF EMS = QF CAF * (NbPartsCAF / NbPartsFiscales) - P / NbPartsFiscales

Pour une personne isolée, on a une pente de 2 (2/1) et pour une personne avec enfant 2,5/1,5 = 1,66.

## Déplacement / modification des QF

In [ ]:
ratio = df["QFEMS"] / df["MONTANT QF CNAF"]

In [ ]:
ratio.describe(percentiles=percentiles)

In [ ]:
ratio[(ratio != 0) * (ratio < 4)].plot.hist(
    bins=50, title="Distribution du rapport QFEMS/QFCAF (hors QFEMS nuls)"
)

In [ ]:
seaborn.ecdfplot(ratio).set(xlim=[0, 4])

In [ ]:
diff = df["QFEMS"] - df["MONTANT QF CNAF"]

In [ ]:
diff.describe(percentiles=percentiles)

In [ ]:
diff[(-1000 < diff) * (diff < 1000)].plot.hist(
    bins=200, title="Distribution de la différence QFEMS-QFCAF"
)

In [ ]:
seaborn.ecdfplot(diff).set(xlim=[-1000, 1000])

In [ ]:
dest = df.rename(
    columns={
        "MONTANT QF CNAF": "CAF",
        "QFEMS ESTIME": "EMS",
        "QFEMS": "EMS",
        "SITUATION FAMILIALE": "TYPOLOGIE",
    }
).drop(columns=["MONTANT QF CNAF C", "QFEMS C"])

In [ ]:
dest.CAF = dest.CAF.round().astype("int64")
dest.EMS = dest.EMS.round().astype("int64")

In [ ]:
dest

In [ ]:
# dest.to_pickle('mapping_qf_caf_ems.pickle')
# dest.to_pickle('mapping_qf_caf_ems_V2.pickle')